In [ ]:
modules = ["selenium","chromedriver_autoinstaller","pymongo","pandas","seaborn","bs4","numpy"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
    print("Instalando módulos")
    for m in modules:
        # para el import quitamos [...] y ==...
        p = m.find("[")
        mi = m if p==-1 else m[:p]
        p = mi.find("==")
        mi = mi if p==-1 else mi[:p]
        torch_loader = importlib.util.find_spec(mi)
        if torch_loader is not None:
            print(m," encontrado")
        else:
            print(m," No encontrado, instalando...",end="")  
            try:        
                r = check_call([sys.executable, "-m", "pip", "install", "--user",  m])
                print("¡hecho!")
            except:
                print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

    print("¡Terminado!")

instala(modules)  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
from datetime import datetime
import time

In [ ]:
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
import sys

def connection(dbname="test",cs = "mongodb://localhost:27017"):
    # Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
    client = MongoClient(cs)
    try:
        s = client.server_info() # si hay error tendremos una excepción
        print("Conectado a MongoDB, versión",s["version"])
        db = client[dbname]
    except:
        e = sys.exc_info()[0]
        print ("connection error")
        print(e)
        db=None
    return db

url = "mongodb+srv://example.mongodb.net/"
db = connection(dbname="example",cs=url)

In [ ]:
import uuid 
unique_id = uuid.getnode() # Obtiene el número MAC de la máquina 

# Fetch any username from the collection
document = db.scraped_data.find_one({}, {"User": 1, "_id": 0})  # Get a single document with the 'User' field
if not document or 'User' not in document:
    raise ValueError("No documents found or 'User' field missing in the collection")

username = document['User']  # Extract the username
username = str(username).replace(" ", "_").replace("/", "_").replace("\\", "_")  # Sanitize username for folder

# Construct the output file path
output_file = f"scraped_data/2024_{username}_{unique_id}.csv"

# Construct the mongoexport command
mongoexport_command = (
    f"mongoexport --uri={url} "
    f"--db=tfg "
    f"--collection=scraped_data "
    f"--type=csv "
    f"--fields=User,Tweet,Date "
    f"--out={output_file}"
)

# Print and run the command
print(f"Running command: {mongoexport_command}")
os.system(mongoexport_command)

In [ ]:
# TO DOWNLOAD JUST THE USERNAMES
import os

# Define output file
output_file = "processed_usernames.csv"

# Construct the mongoexport command
mongoexport_command = (
    f"mongoexport --uri={url} "
    f"--db=example "
    f"--collection=processed2024 "  # Ensure this is the correct collection name
    f"--type=csv "
    f"--fields=_id "  # Export only the _id field
    f"--out={output_file}"
)

# Print and execute the command
print(f"Running command: {mongoexport_command}")
os.system(mongoexport_command)

In [ ]:
db.scraped_data.drop()